### Insertion Sort

Есть отсортированная часть (считаем, что один элемент отсортирован), а есть неотсортированная. Берем последующий элемент $key$ в неотсортированной части и сравниваем его с элементами из отсортированной части. Сдвигаем их вправо и вставляем $key$ на освободившееся место.

* $O(n^2)$ - avg and worst time
* $O(1)$ - memory
* **Stable**, так как мы не меняем элементы местами, а сплошняком слева направо собираем отсортированный массив. При вставке нового элемента на свое место он встает правее элементов с теми же ключами (не заходим в цикл while при равенстве ключей).

In [14]:
def insertion_sort(arr):
    for j in range(1, len(arr)):
        key = arr[j]
        # Вставляем arr[j] в отсортированную часть arr[0, ..., j - 1]
        i = j - 1
        while i >= 0 and arr[i] > key:
            arr[i + 1] = arr[i]
            i -= 1
        arr[i + 1] = key
        
    return arr

In [15]:
arr = [9, 7, 8, 8, 5, 7, 2, 0, -3]
insertion_sort(arr)

[-3, 0, 2, 5, 7, 7, 8, 8, 9]

### Selection Sort

Для каждого элемента $i$ массива находим минимальный элемент $min\_idx$ справа от него и меняем их местами, тем самым ставя на место минимальный элемент. Грубо говоря мы каждый раз заново находим минимум в неотсортированной части массива.

* $O(n^2)$ - avg and worst time
* $O(1)$ - memory
* **Unstable**, так как при поиске минимального элемента мы берём указатель *последнего* (так как проходимся до конца массива) и ставим его перед остальными минимальными элементами. 

In [5]:
def selection_sort(arr):
    for i in range(len(arr)):
        min_idx = i
        for j in range(i + 1, len(arr)):
            if arr[j] < arr[min_idx]:
                min_idx = j
        arr[i], arr[min_idx] = arr[min_idx], arr[i]
        
    return arr

In [6]:
arr = [9, 7, 8, 8, 5, 4, 2, 0, 1]
selection_sort(arr)

[0, 1, 2, 4, 5, 7, 8, 8, 9]

### Merge Sort

* $O(n\, log(n))$ - avg and worst time
* $O(n)$ - memory
* **Stable**, так как при рекурсивный вызов MergeSort не меняет местами элементы с одинаковым значением. Процедура Merge для элементов с одинаковым значением сначала берет элемент из левого поддерева (для ascending сортировки), поэтому порядок также не нарушается.

In [112]:
def merge(a, b):
    i, j = 0, 0
    merged = []
    
    while i < len(a) and j < len(b):
        if a[i] <= b[j]:
            merged.append(a[i])
            i += 1
        else:
            merged.append(b[j])
            j += 1
            
    merged += b[j:]
    merged += a[i:]
    
    return merged


def mergeSort(a):
    if len(a) > 1:
        m = len(a) // 2
        return merge(mergeSort(a[:m]), 
                     mergeSort(a[m:]))
    return a

In [58]:
a = [2, 3, 13]
b = [4, 7, 9, 12, 14, 24]
merge(a, b)

[2, 3, 4, 7, 9, 12, 13, 14, 24]

In [113]:
a = [8, 7, 1, 10, 24, 2, 5, 8, 0]
mergeSort(a)

[0, 1, 2, 5, 7, 8, 8, 10, 24]

In [1]:
def bad_merge(arr, l, m, r):
    L, R = arr[l:m], a[m:r]
    k, i, j = 0, 0, 0
    
    while i < len(L) and j < len(R):
        if L[i] <= R[j]:
            arr[k] = L[i]
            i += 1
        else:
            arr[k] = R[j]
            j += 1
        k += 1
    
    arr += R[j:]
    arr += L[i:]

In [56]:
def bad_mergeSort(a, l, r):
    if l < r:
        m = (l + r) // 2
        return merge(mergeSort(a, l, m), 
                     mergeSort(a, m + 1, r))
    return [a[l]]

In [65]:
import heapq

def bad_iterativeMergeSort(a):
    queue = []
    for i in range(len(a)):
        heapq.heappush(queue, [a[i]])

    while len(queue) > 1:
        merged = merge(heapq.heappop(queue), heapq.heappop(queue))
        heapq.heappush(queue, merged)
        print(queue)
    return heapq.heappop(queue)

a = [7, 6, 5, 4, 3, 2, 1]
bad_iterativeMergeSort(a)

[[1, 2], [4], [3], [7], [5], [6]]
[[1, 2, 3], [4], [6], [7], [5]]
[[1, 2, 3, 4], [5], [6], [7]]
[[1, 2, 3, 4, 5], [7], [6]]
[[1, 2, 3, 4, 5, 6], [7]]
[[1, 2, 3, 4, 5, 6, 7]]


[1, 2, 3, 4, 5, 6, 7]

### Heap Sort

Использует структуру max кучи для сортировки. Строим кучу из массива и ставим ее корень в конец массива, как максимальный элемент. Так как в корне теперь новый элемент поддерживаем свойство max кучи в неотсортированной части и итеративно строим отсортированный массив с конца. 

* $O(n\, log(n))$ - avg and worst time
* $O(1)$ - memory (inplace realization)
* **Unstable**, так как корневой элемент в куче, стоящий на первом месте встает в конец массива. Допустим, если следующий элемент после корневого имеет тот же ключ, то он встанет в отсортированном массиве перед корневым.

In [109]:
def max_sift_down(a, i, h_size):  # O(log(n))
    while 2 * i + 1 <= h_size - 1:  # пока левый ребенок лежит в куче
        l = 2 * i + 1
        r = 2 * i + 2
        largest = l
        if r <= h_size - 1 and a[r] > a[l]:  # если есть правый ребенок в куче
            largest = r
        if a[i] >= a[largest]:
            break
        a[largest], a[i] = a[i], a[largest]
        i = largest

def build_max_heap(arr):  # O(n)
    for i in range(len(arr) // 2, -1, -1):
        max_sift_down(arr, i, len(arr))

def heap_sort(arr):
    build_max_heap(arr)
    h_size = len(arr)
    
    for _ in range(len(arr) - 1):  # O(n * log(n))
        arr[0], arr[h_size - 1] = arr[h_size - 1], arr[0]
        h_size -= 1
        max_sift_down(arr, 0, h_size)  # O(log(n))

In [110]:
arr = [9, 2, 7, 2, 5, 0, 1, 8, 5]
build_max_heap(arr)
arr

[9, 8, 7, 5, 5, 0, 1, 2, 2]

In [111]:
arr = [9, 2, 7, 2, 5, 0, 1, 8, 5]
heap_sort(arr)
arr

[0, 1, 2, 2, 5, 5, 7, 8, 9]

### QuickSort

Сортировка методом разделяй и влавствуй. Выбираем опорный элемент и проходимся по массиву, поддерживая две области: от нуля до $i - 1$ для элементов меньше опорного, от $i$ до $j$ - больше либо равных. Рекурсивно вызываем `quick_sort` на обоих областях пока $p$ и $r$ не сравнятся.
* $O(n\, log(n))$ - avg time, $O(n^2)$ - worst time
* $O(1)$ - memory
* **Unstable**

In [115]:
def partition(arr, p, r):
    x = arr[r]
    i = p
    # k = p
    for j in range(p, r):
        if arr[j] < x:
            arr[i], arr[j] = arr[j], arr[i]
            i += 1
    arr[r], arr[i] = arr[i], arr[r]
    return i


def quick_sort(arr, p, r):
    if p < r:
        q = partition(arr, p, r)
        quick_sort(arr, p, q - 1)
        quick_sort(arr, q + 1, r)

In [119]:
arr = [9, 2, 7, 2, 5, 0, 1, 8, 5]
print(arr)

q = partition(arr, 0, len(arr) - 1)
print(arr[:q], arr[q], arr[q+1:])

[9, 2, 7, 2, 5, 0, 1, 8, 5]
[2, 2, 0, 1] 5 [7, 9, 8, 5]


In [4]:
arr = [9, 2, 7, 2, 5, 0, 1, 8, 5]
quick_sort(arr, 0, len(arr) - 1)
arr

[0, 1, 2, 2, 5, 5, 7, 8, 9]

Если в массиве много одинаковых чисел, то стоит использовать 3 разбиения, а не 2, не рассматривая элементы с одинаковым значением.

In [5]:
import random

def quick_sort3(nums):
    if len(nums) <= 1:
        return nums
    q = random.choice(nums)
    s_nums = []
    m_nums = []
    e_nums = []
    for n in nums:
        if n < q:
            s_nums.append(n)
        elif n > q:
            m_nums.append(n)
        else:
            e_nums.append(n)
    return quick_sort3(s_nums) + e_nums + quick_sort3(m_nums)

In [11]:
arr = []
for i in range(100):
    arr += [i] * 1000

In [12]:
%%time
quick_sort(arr, 0, len(arr) - 1)

CPU times: total: 2.47 s
Wall time: 2.47 s


In [13]:
%%time
s = quick_sort3(arr)

CPU times: total: 46.9 ms
Wall time: 46.9 ms


### Count Sort

Подходит для массивов с ограниченным числом ключей $m$. Создаем массив $B$, который подсчитывает уникальные элементы в $A$. Кумулятивно складывая количество вхождений получаем индексы последнего уникального элемента в отсортированном массиве $A_s$. Проходим по изначальному массиву с конца, получаем индекс элемента $B[a]$ и вставляем на это место в отсортированном массиве.

* $O(n + m)$ - avg and worst time
* $O(n + m)$ - memory
* **Stable**, так как уникальные элементы вставляются в массив справа налево, поэтому по изначальному массиву тоже проходим справа налево. Стабильность важна для использования `count_sort` в поразрядной сортировке.

In [7]:
def count_sort(arr, m):
    brr = [0] * m
    s_arr = [0] * len(arr)
    for j in range(len(arr)):
        a = arr[j]  # уникальный элемент в arr/ индекс в массиве brr
        brr[a] += 1
    for a in range(1, m):
        brr[a] = brr[a] + brr[a - 1]
    for j in range(len(arr) - 1, -1, -1):
        a = arr[j]
        s_arr[brr[a] - 1] = a
        brr[a] -= 1
        
    return s_arr

In [8]:
arr =[]
for i in range(5):
    for j in range(5):
        arr += [j] * i
print(arr)

[0, 1, 2, 3, 4, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]


In [10]:
print(count_sort(arr, 5))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


### Radix Sort

Сортировка подсчетом вызывается для каждого из $d$ разрядов, начиная с единичного. Благодаря стабильности `count_sort` последующих разрядов не перемешивает предыдущие.

* $O(d n)$ - avg and worst time
* $O(n)$ - memory
* **Stable**

In [18]:
def countingSortForRadix(inputArray, placeValue):
    # placeValue - digit position in the number (1, 10, 100, ...)
    countArray = [0] * 10
    inputSize = len(inputArray)
    
    for i in range(inputSize): 
        # if given number = 123, placeValue = 10, then placeElement = 2
        placeElement = (inputArray[i] // placeValue) % 10
        countArray[placeElement] += 1

    for i in range(1, 10):
        countArray[i] += countArray[i-1]
        
    # Reconstructing the output array
    outputArray = [0] * inputSize
    i = inputSize - 1
    while i >= 0:
        currentEl = inputArray[i]
        placeElement = (currentEl // placeValue) % 10
        countArray[placeElement] -= 1
        
        newPosition = countArray[placeElement]
        outputArray[newPosition] = currentEl
        i -= 1
        
    return outputArray


def radixSort(array):
    # Step 1 -> Find the maximum element in the input array
    maxEl = max(array)
    
    # Step 2 -> Find the number of digits in the `max` element
    D = 1
    while maxEl > 0:
        maxEl /= 10
        D += 1
    
    # Step 3 -> Initialize the place value to the least significant place
    placeVal = 1
    while D > 0:
        array = countingSortForRadix(array, placeVal)
        placeVal *= 10  
        D -= 1
        
    return array

In [27]:
arr = [i for i in range(120, 95, -1)]
print(arr)

[120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96]


In [28]:
print(radixSort(arr))

[96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120]
